# Model 1: meta-llama/Meta-Llama-3-8B-Instruct

In [4]:
!pip install -U accelerate transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.0 MB/s eta 0:00:00


In [8]:
!pip install -i https://pypi.org/simple/ bitsandbytes --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00


In [13]:
!pip install accelerate --q

In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoConfig, BitsAndBytesConfig

In [2]:
q_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_quant_type="nf4",
  )

In [3]:
def create_model_tokenizer(name, q_config):
  tokenizer = AutoTokenizer.from_pretrained(name)
  model = AutoModelForCausalLM.from_pretrained (
      name,
      quantization_config = q_config,
      device_map="cuda",
      torch_dtype="auto",
      trust_remote_code = True
  )
  return model, tokenizer

In [4]:
# Load model directly
model1, tokenizer1 = create_model_tokenizer("NousResearch/Meta-Llama-3-8B-Instruct", q_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [13]:
nli_pipeline = pipeline(
    "text2text-generation",
    model=model1,
    tokenizer=tokenizer1,
    # device=0  # Set the device to 0 if you want to use GPU, or -1 for CPU
)

The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [7]:
def paraphrase_check(question1, question2, model, tokenizer):
  """
  This function checks if two questions are paraphrases using a pre-trained LLM.

  Args:
    question1: The first question string.
    question2: The second question string.
    model: The loaded LLM model.
    tokenizer: The tokenizer corresponding to the LLM.

  Returns:
    A boolean indicating if the questions are paraphrases.
  """
  # Encode both questions with padding
  inputs = tokenizer([question1, question2], padding="max_length", max_length=512, return_tensors="pt")  # Adjust max_length as needed

  # Generate outputs for each question
  outputs = model(**inputs)
  last_hidden_states = outputs.last_hidden_state

  # Get the CLS token hidden state for each question
  question1_embedding = last_hidden_states[:, 0, :]
  question2_embedding = last_hidden_states[:, 1, :]

  # Calculate cosine similarity between the embeddings
  similarity = torch.nn.functional.cosine_similarity(question1_embedding, question2_embedding)

  # Set a threshold for similarity (adjust as needed)
  threshold = 0.8

  return similarity.item() > threshold

In [10]:
question1 = "What is the capital of France?"
question2 = "Can you tell me the capital city of France?"
tokenizer1.add_special_tokens({'pad_token': '[PAD]'})
if paraphrase_check(question1, question2, model1, tokenizer1):
    print("The questions are paraphrases.")
else:
  print("The questions are not paraphrases.")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
